In [ ]:
!pip install -qU llama_index sqlalchemy pandas
!pip install openai

Importing required libraries

In [7]:
from google.colab import userdata
import openai
import os
import pandas as pd
from sqlalchemy import create_engine
from llama_index import SQLDatabase
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.agent import OpenAIAgent


In [ ]:
# os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
# openai_api_key= userdata.get('OPENAI_API_KEY')

In [9]:
df = pd.read_csv("/content/data.csv",encoding='latin1')

In [ ]:
df.head()


Create SQLAlchemy engine with SQLlite and convert dataframe into SQL tables

In [13]:
engine = create_engine('sqlite://',echo=False)
df.to_sql('reviews', con=engine)

301296

In [14]:
reviews_db= SQLDatabase(engine,include_tables=['reviews'])

Instantiate the LLM and servicecontext

In [16]:
llm = OpenAI(model_name="gpt-4-0613",
             temperature=0,
             streaming=True)
embedding =OpenAIEmbedding()
servicecontext= ServiceContext.from_defaults(llm=llm,
                                             embed_model=embedding,
                                             chunk_size=500)

In [17]:
sql_query_engine = NLSQLTableQueryEngine(sql_database=reviews_db,
                                         tables=['reviews'],
                                         service_context=servicecontext)

In [18]:
description = ("Provides details information")

In [20]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                                         name="sql_query",
                                         description=description)

agent= OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)

Generate the response, you can write your input in the brackets to get the response

In [ ]:
response = agent.chat("How many unique countries are there")